In [13]:
import time, psutil, os
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
import awkward as ak
from collections import Counter
from statsmodels.stats.weightstats import DescrStatsW
import csv

In [14]:
train_dataset = h5py.File('hgcal_electron_data_0001.h5', "r")
nhits = np.array(train_dataset['nhits'])
target_energy = np.array(train_dataset['target'])

In [15]:
uniq_z = [13.8774995803833, 14.767499923706055, 16.782499313354492, 17.672500610351562, 19.6875, 20.577499389648438, 22.6924991607666, 23.582500457763672, 25.697500228881836, 26.587499618530273, 28.702499389648438, 29.592500686645508, 31.50749969482422, 32.397499084472656, 34.3125, 35.20249938964844, 37.11750030517578, 38.00749969482422, 39.92250061035156, 40.8125, 42.907501220703125, 44.037498474121094, 46.412498474121094, 47.54249954223633, 49.68199920654297, 50.6879997253418, 52.881500244140625, 53.903499603271484]

In [16]:
csvLabel = []
for i in range(len(uniq_z)):
    csvLabel.append("energySum_layer" + str(i))
for i in range(len(uniq_z)):
    csvLabel.append("xMean_layer" + str(i))
for i in range(len(uniq_z)):
    csvLabel.append("xStd_layer" + str(i))
for i in range(len(uniq_z)):
    csvLabel.append("yMean_layer" + str(i))
for i in range(len(uniq_z)):
    csvLabel.append("yStd_layer" + str(i))
for i in range(len(uniq_z)):
    csvLabel.append("rMean_layer" + str(i))
for i in range(len(uniq_z)):
    csvLabel.append("rStd_layer" + str(i))
for i in range(len(uniq_z)):
    csvLabel.append("nhits_layer" + str(i))
csvLabel.append("nhits_total")
csvLabel.append("target_energy")

In [11]:
#print(csvLabel)
print(len(csvLabel))

226


In [12]:
outFile = "hgcal_electron_data_0001.csv"
with open(outFile, mode="w", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(csvLabel)
    
# Current index in the rechit_energy array
current_index = 0
events = 0
maxEvents = -1

# Iterate over the nhits array
for nhit in nhits:
    if events >= maxEvents and maxEvents > 0:
        break
    # Convert nhit to an integer
    nhit = int(nhit)
    # Extract the corresponding subarray from rechit_energy
    subArray_energy = train_dataset['rechit_energy'][current_index:current_index + nhit] * 0.1
    subArray_z = train_dataset['rechit_z'][current_index:current_index + nhit]
    subArray_x = train_dataset['rechit_x'][current_index:current_index + nhit]
    subArray_y = train_dataset['rechit_y'][current_index:current_index + nhit]
    
    subArray_r = np.array([(x**2 + y**2)**0.5 for x,y in zip(subArray_x,subArray_y)])
    
    energy_perZ = [ subArray_energy[subArray_z == zVal] for zVal in uniq_z]
    r_perZ = [ subArray_r[subArray_z == zVal] for zVal in uniq_z]
    x_perZ = [ subArray_x[subArray_z == zVal] for zVal in uniq_z]
    y_perZ = [ subArray_y[subArray_z == zVal] for zVal in uniq_z]
    count_perZ = [len(val) for val in r_perZ]
    
    energySum_perZ = [sum(x) for x in energy_perZ]
    
    x_DescrStatsW = [ DescrStatsW(array, weights=weights, ddof=0) for array,weights in zip(x_perZ,energy_perZ) ]
    xMean_perZ = [ stat.mean for stat in x_DescrStatsW ]
    xMean_perZ = [0 if np.isnan(value) else value for value in xMean_perZ]
    xStd_perZ = [ stat.std for stat in x_DescrStatsW ]
    xStd_perZ = [0 if np.isnan(value) else value for value in xStd_perZ]
    
    y_DescrStatsW = [ DescrStatsW(array, weights=weights, ddof=0) for array,weights in zip(y_perZ,energy_perZ) ]
    yMean_perZ = [ stat.mean for stat in y_DescrStatsW ]
    yMean_perZ = [0 if np.isnan(value) else value for value in yMean_perZ]
    yStd_perZ = [ stat.std for stat in y_DescrStatsW ]
    yStd_perZ = [0 if np.isnan(value) else value for value in yStd_perZ]
    
    r_DescrStatsW = [ DescrStatsW(array, weights=weights, ddof=0) for array,weights in zip(r_perZ,energy_perZ) ]
    rMean_perZ = [ stat.mean for stat in r_DescrStatsW ]
    rMean_perZ = [0 if np.isnan(value) else value for value in rMean_perZ]
    rStd_perZ = [ stat.std for stat in r_DescrStatsW ]
    rStd_perZ = [0 if np.isnan(value) else value for value in rStd_perZ]
    
    if events == 0:   
        print(len(energySum_perZ + xMean_perZ + xStd_perZ + yMean_perZ + yStd_perZ + rMean_perZ + rStd_perZ + count_perZ + [nhit] + [target_energy[events]]))
    
    with open(outFile, mode="a", newline='') as f:
        writer = csv.writer(f)
        writer.writerow(energySum_perZ + xMean_perZ + xStd_perZ + yMean_perZ + yStd_perZ + rMean_perZ + rStd_perZ + count_perZ + [nhit] + [target_energy[events]])
        
    current_index += nhit
    events += 1


226


In [ ]:
len(target_energy)